In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from tqdm import tqdm

In [2]:
def calc_hhi(arr):
    sum1 = np.sum(arr)
    arr1 = (100*arr)/sum1
    arr2 = arr1*arr1
    return np.sum(arr2)

In [3]:
df_int2 = pd.read_csv("total_pickups_dropoffs_cluster_id_250m_at_intersections.csv")

df_groupby = df_int2.groupby(by = ['cluster_id'])
unique_cluster_ids = df_int2['cluster_id'].unique()

type_array = ['all', 'morning', 'late_morning', 'afternoon', 'evening', 'late_evening']

data = {}

for type1 in type_array:
    
    cid_list_phhi = []
    cid_list_dhhi = []
    cid_ratio = []
    cid_total_pickups = []
    cid_total_dropoffs = []
    cluster_length = []
    for cluster_id in unique_cluster_ids:
        temp_df = df_groupby.get_group(cluster_id)
        cid_list_phhi.append(calc_hhi(temp_df[type1 + '_pickups'].values))
        cid_list_dhhi.append(calc_hhi(temp_df[type1 + '_dropoffs'].values))
        cid_ratio.append(calc_hhi(temp_df[type1 + '_pickups'].values)/calc_hhi(temp_df[type1 + '_dropoffs'].values))
        cid_total_pickups.append(np.sum(temp_df[type1 + '_pickups'].values))
        cid_total_dropoffs.append(np.sum(temp_df[type1 + '_dropoffs'].values))
        cluster_length.append(len(temp_df))
    
    if type1 == 'all':
        data['cluster_ids'] = unique_cluster_ids
        data['c_len'] = cluster_length
    data[type1 + '_c_pickups'] = cid_total_pickups
    data[type1 + '_c_dropoffs'] = cid_total_dropoffs
    data[type1 + '_phhi'] = cid_list_phhi
    data[type1 + '_dhhi'] = cid_list_dhhi
    data[type1 + '_hhi_ratio'] = cid_ratio


In [4]:
df_hhi_final = pd.DataFrame(data = data)
df_hhi_final.to_csv("cluster_hhi_results.csv")

In [5]:
df_hhi_final.head(5)

,cluster_ids,c_len,all_c_pickups,all_c_dropoffs,all_phhi,all_dhhi,all_hhi_ratio,morning_c_pickups,morning_c_dropoffs,morning_phhi,...,evening_c_pickups,evening_c_dropoffs,evening_phhi,evening_dhhi,evening_hhi_ratio,late_evening_c_pickups,late_evening_c_dropoffs,late_evening_phhi,late_evening_dhhi,late_evening_hhi_ratio
0,2947,4,124375,101045,3767.241240,3412.456331,1.103968,5543,5294,4194.641608,...,30237,19838,3778.731730,3373.926496,1.119980,9450,7102,4150.426696,3750.000892,1.106780
1,1933,7,73183,64585,2895.485211,2705.334543,1.070287,2937,2574,4025.132713,...,20190,17940,2974.461241,2712.634025,1.096521,8269,7251,2860.500722,2833.223409,1.009628
2,2384,5,82080,70302,3097.412372,2905.793864,1.065944,2003,4091,3453.444397,...,19316,10512,3144.917163,2943.700712,1.068355,3673,2470,3435.632194,3316.500844,1.035921
3,2000,8,132099,98439,1717.288266,1603.424987,1.071013,5436,5229,1811.292931,...,33252,21208,1701.856138,1591.744478,1.069177,9376,5971,1778.216012,1628.819706,1.091721
4,2165,8,66529,67387,3136.706273,2835.572086,1.106199,2949,4037,2728.524627,...,14211,11866,2706.592655,2461.332284,1.099645,3012,2992,2580.408777,2299.652907,1.122086
